In [1]:
# bibliotecas
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
import numpy as np
import torch

from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, models

from sklearn.metrics import classification_report, confusion_matrix, f1_score, roc_auc_score
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split

from torch.utils.data import Subset, DataLoader, ConcatDataset, random_split
from collections import Counter

### Carregamento e Pré-Processamento do Dataset

In [ ]:
from utils import data_utils

image_size = (224, 224)
train_data, test_data = data_utils.load_and_preprocess_data(resize_to=image_size, keras_format=True)

### Divisão Holdout

In [ ]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)
train_idx, val_idx = next(split.split(np.zeros(len(train_data.targets)), train_data.targets))

In [ ]:
train_split = Subset(train_data, train_idx)
val_split = Subset(train_data, val_idx)

In [ ]:
print(f"Treino:    {len(train_split)} imagens")
print(f"Validação: {len(val_split)} imagens")
print(f"Teste:     {len(test_data)} imagens")

In [ ]:
def get_X_y(dataset):
    X = []
    y = []
    for img, label in dataset:
        X.append(img.numpy())  
        y.append(label)
    X = np.stack(X)
    y = np.array(y)
    return X, y

In [ ]:
X_train, y_train = get_X_y(train_split)
X_val, y_val = get_X_y(val_split)
X_test, y_test = get_X_y(test_data)

## Treino

### Modelo

**ResNet50**

O ResNet50 é uma arquitetura de rede neural convolucional profunda de ponta, desenvolvida pela Microsoft Research em 2015. É uma variante da popular arquitetura ResNet e compreende 50 camadas que permitem o aprendizado de arquiteturas muito mais profundas do que era possível anteriormente, sem o problema de gradientes que desaparecem. 

A arquitetura do ResNet50 é dividida em quatro partes principais: as camadas convolucionais, o bloco de identidade, o bloco convolucional e as camadas totalmente conectadas. As camadas convolucionais são responsáveis ​​por extrair características da imagem de entrada, o bloco de identidade e o bloco convolucional processam e transformam essas características, e as camadas totalmente conectadas fazem a classificação final. O ResNet50 foi treinado no grande conjunto de dados ImageNet, alcançando uma taxa de erro equivalente ao desempenho humano, tornando-o um modelo poderoso para diversas tarefas de classificação de imagens, como detecção de objetos, reconhecimento facial e análise de imagens médicas. Além disso, também tem sido usado como um extrator de características para outras tarefas, como detecção de objetos e segmentação semântica.

Fonte: https://medium.com/@nitishkundu1993/exploring-resnet50-an-in-depth-look-at-the-model-architecture-and-code-implementation-d8d8fa67e46f

In [ ]:
device = "GPU" if len(tf.config.list_physical_devices('GPU')) > 0 else "CPU"
print(f"Usando: {device}")

In [ ]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(*image_size, 3))
base_model.trainable = False  # pra transfer Learning congela base

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=10,
    batch_size=32,
    verbose=1
)

In [ ]:
y_pred_prob = model.predict(X_test).ravel()
y_pred = (y_pred_prob > 0.5).astype(int)

cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(5, 4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Real', 'Fake'], yticklabels=['Real', 'Fake'])
plt.xlabel("Predito")
plt.ylabel("Real")
plt.title("Matriz de Confusão")
plt.show()

print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred, target_names=["Real", "Fake"]))

roc_auc = roc_auc_score(y_test, y_pred_prob)
print(f"AUC (ROC): {roc_auc:.4f}")

plt.figure(figsize=(8, 4))
plt.plot(history.history['accuracy'], label='Treino')
plt.plot(history.history['val_accuracy'], label='Validação')
plt.title("Curva de Aprendizado - Acurácia")
plt.xlabel("Época")
plt.ylabel("Acurácia")
plt.legend()
plt.grid()
plt.show()

plt.figure(figsize=(8, 4))
plt.plot(history.history['loss'], label='Treino')
plt.plot(history.history['val_loss'], label='Validação')
plt.title("Curva de Aprendizado - Perda")
plt.xlabel("Época")
plt.ylabel("Perda (Loss)")
plt.legend()
plt.grid()
plt.show()
